In [1]:
from PIL import Image
from PIL import ImageDraw
import os
import detect
import tflite as tf
import platform
import datetime
import cv2
import time
import numpy as np
import io
from io import BytesIO
from flask import Flask, request, Response, jsonify
import random
import re
import base64

ModuleNotFoundError: No module named 'tflite'

In [3]:
!pip install tensorflow

                                              0.0/272.9 MB ? eta -:--:--
                                              0.0/272.9 MB ? eta -:--:--
                                              0.0/272.9 MB ? eta -:--:--
                                            0.0/272.9 MB 217.9 kB/s eta 0:20:53
                                            0.0/272.9 MB 219.4 kB/s eta 0:20:44
                                            0.1/272.9 MB 273.8 kB/s eta 0:16:37
                                            0.1/272.9 MB 327.3 kB/s eta 0:13:54
                                            0.1/272.9 MB 386.4 kB/s eta 0:11:46
                                            0.2/272.9 MB 468.3 kB/s eta 0:09:43
                                            0.2/272.9 MB 541.9 kB/s eta 0:08:24
                                            0.3/272.9 MB 655.2 kB/s eta 0:06:57
                                            0.4/272.9 MB 791.2 kB/s eta 0:05:45
                                            0.5/272.9 MB 921.

ERROR: Could not install packages due to an OSError: [WinError 2] Das System kann die angegebene Datei nicht finden: 'C:\\Python311\\Scripts\\tensorboard.exe' -> 'C:\\Python311\\Scripts\\tensorboard.exe.deleteme'



In [79]:
folder_path = r'C:\Users\maxsc\Desktop\TU\DIC3\Pictures_small'
images_s = []

# Iterate over each file in the folder
for filename in os.listdir(folder_path):
    # Construct the full file path
    file_path = os.path.join(folder_path, filename)

    # Check if the file is a JPEG image
    if os.path.isfile(file_path) and filename.lower().endswith(".jpg"):
        with open(file_path, "rb") as file:
            # Read the image data
            image_data = file.read()

            # Encode the image data as base64
            encoded_data = base64.b64encode(image_data).decode("utf-8")

            # Append the encoded string to the list
            images_s.append(encoded_data)

print(len(images_s))


296


In [80]:
#Preprocess an image


example = images_s[25]
image_bytes = base64.b64decode(example)
image = Image.open(io.BytesIO(image_bytes))
image = image.resize((300, 300))  # Resize to match the model's input size
image_np = np.array(image)  # Convert PIL image to NumPy array
image_np_expanded = np.expand_dims(image_np, axis=0)



In [69]:
#Train interpreter model NOT READY

module_path = r"C:\Users\maxsc\Desktop\TU\DIC3\lite-model_ssd_mobilenet_v2_100_fp32_default_1.tflite"
interpreter = tf.Interpreter(model_path=model_path)

AttributeError: module 'tflite' has no attribute 'Interpreter'

In [ ]:
interpreter = tf.lite.Interpreter(model_path='path/to/your/model.tflite')
interpreter.allocate_tensors()

#Example image
input_tensor = interpreter.get_input_details()[0]['index']
interpreter.set_tensor(input_tensor, image_np_expanded)


interpreter.invoke()

In [ ]:

# Get the output tensors
output_details = interpreter.get_output_details()
boxes = interpreter.get_tensor(output_details[0]['index'])
classes = interpreter.get_tensor(output_details[1]['index'])
scores = interpreter.get_tensor(output_details[2]['index'])
num_detections = int(output_details[3]['index'][0])

In [ ]:
# Filter out detections with low confidence
threshold = 0.5
detections = []
for i in range(num_detections):
    if scores[0][i] > threshold:
        class_id = int(classes[0][i])
        class_name = label_map[class_id]
        box = boxes[0][i]
        detection = {
            'class': class_name,
            'score': float(scores[0][i]),
            'box': [float(box[0]), float(box[1]), float(box[2]), float(box[3])]
        }
        detections.append(detection)